In [6]:
from metaflow import Metaflow, Flow, Run, Step, Task, DataArtifact

In [7]:
Metaflow().flows

[Flow('ModelFlow')]

In [8]:
flow = Metaflow()['ModelFlow']


In [9]:
import pandas as pd
from datetime import datetime, date, time, timedelta

def to_datetime(str_date):
    type(str_date)
    if(str_date != None):
        str_date = str_date.replace('T', ' ').replace('.fZ', '')
        return datetime.strptime(str_date, '%Y-%m-%d %H:%M:%S')
    return None

def duration(init_date, end_date):
    if(end_date != None):
        delta = end_date - init_date
        return delta.total_seconds() / 60 / 60
    return -1



In [11]:
flow_name = 'ModelFlow'
flow = Metaflow()[flow_name]
flow_runs = {}
columns = ['run_id', 'version_name', 'created_at', 'finished_at', 'run_duration', 'scores_mean', 'scores_std']

for idx, run in enumerate(list(flow)):
    if(run.finished):
        scores = run['build_model'].task.data.scores.T
        scores_mean= scores.values[0].mean()
        scores_std = scores.values[0].std()

        version_name = run['start'].task.data.version_name
        created_at = to_datetime(run.created_at)
        finished_at = to_datetime(run.finished_at)
        run_duration = duration(created_at, finished_at)

        values = [run.id, version_name, created_at, finished_at, run_duration, scores_mean, scores_std]
        flow_runs[idx] = values
    
flow_runs = pd.DataFrame.from_dict(flow_runs, orient='index', columns = columns)   
flow_runs

,run_id,version_name,created_at,finished_at,run_duration,scores_mean,scores_std
0,1581734961379775,sample_1%,2020-02-15 02:49:21,2020-02-15 04:35:55,1.776111,0.934574,0.057917
1,1581734723626575,sample_1%,2020-02-15 02:45:23,2020-02-15 02:46:06,0.011944,0.905300,0.139863
2,1581734470978840,sample_1%,2020-02-15 02:41:10,2020-02-15 02:43:38,0.041111,0.878445,0.126454
5,1581734070542262,sample_1%,2020-02-15 02:34:30,2020-02-15 02:35:20,0.013889,0.880806,0.146776
18,1581635431321592,sample_10%,2020-02-13 23:10:31,2020-02-13 23:12:06,0.026389,0.910607,0.101072
